In [ ]:
from glob import glob
import os
# tsv files

tsv_files = glob('/home/DeepPhonemizer/tsv/*.tsv')

In [ ]:
all_languages = [x.split("/")[-1].split("_")[0] for x in tsv_files]
set_languages = list(set(all_languages))
print(set_languages)

In [ ]:
from glob import glob

tsv_files = glob("/home/DeepPhonemizer/tsv/*.tsv")
train_data = []
all_chars = set()
all_phonemes = set()

for f in tsv_files:
    language_code = f.split("/")[-1].split("_")[0]
    with open(
        f,
        "r",
        encoding="utf-8",
    ) as f:
        lines = f.readlines()
    # Prepare data as tuples (lang, word, phoneme)
    lines = [l.replace(" ", "").replace("\n", "") for l in lines]
    splits = [l.split("\t") for l in lines]
    for grapheme, phoneme in splits:
        if len(grapheme) > 0 and len(phoneme) > 0:
            all_chars.update(grapheme)
            all_phonemes.update(phoneme)
            train_data.append((language_code, grapheme, phoneme))

In [ ]:
# take 0.01 (max 10) of the data from train_grouped_by_lang per language for validation and remove it from train
import random
from collections import defaultdict

train_grouped_by_lang = defaultdict(list)
validate_grouped_by_lang = defaultdict(list)

for lang, word, phoneme in train_data:
    train_grouped_by_lang[lang].append((word, phoneme))

for lang, data in train_grouped_by_lang.items():
    random.shuffle(data)
    n = min(1000, int(len(data) * 0.01))
    validate_grouped_by_lang[lang].extend(data[:n])
    train_grouped_by_lang[lang] = data[n:]


In [ ]:
train_data = []
validation_data = []
for lang, data in train_grouped_by_lang.items():
    for grapheme, phoneme in data:
        train_data.append((lang, grapheme, phoneme))
        
for lang, data in validate_grouped_by_lang.items():
    for grapheme, phoneme in data:
        validation_data.append((lang, grapheme, phoneme))

In [ ]:
print("".join(list(all_chars)))

In [ ]:
print(list(all_phonemes))

In [ ]:
from dp.phonemizer import Phonemizer

if __name__ == "__main__":

    checkpoint_path = "/workspace/pretrained_models/g2p/checkpoints/best_model_no_optim.pt"
    phonemizer = Phonemizer.from_checkpoint(checkpoint_path)

    sentences = [
        "Aus einem fernen Land, von dem du wahrscheinlich noch nie etwas gehört hast, Marvin.",
        "Moment, woher kennst du meinen Namen?",
        "Ich bin eine gute Zuhörerin und hier unter Deck ist es nicht gerade geräumig.",
        "Du scheinst ein netter Junge zu sein. Ich habe ein Angebot für dich.",
        "Siehst du den Mistkerl, der die Pfeile schnitzt? Dieses Arschloch hat meine Schatulle genommen und sie bei seiner Beute versteckt. Ich habe gesehen, wie er damit im Frachtraum verschwunden ist.",
        "Ich dachte, er würde meine Schatulle in einer Truhe verstecken, aber dann konnte ich das laute Knarzen von Schiffsplanken hören. Könntest du dich im Frachtraum mal umsehen und schauen, ob es dort ein Versteck gibt?",
        "Ich habe deine Schatulle gefunden.",
        "Das ist ja grossartig! Vielen Dank, mein Junge!",
        "Im Moment habe ich leider keine Möglichkeit dich für deine Hilfe zu bezahlen. Aber ich verspreche dir, sobald wir auf Archolos sind sorge ich dafür, dass meine Freunde dich entlohnen!"
        "Sieht aus, als wären wir bald da.",
        "Die paar zusammengestückelten Minecrawler-Platten haben im underground Tempel einfach zu viele Hits durchgelassen, man!",
        "Woher willst du das wissen?",
        "WAS SOLL DAS?",
        "Das spiel heisst Gothic.",
    ]



    for text in sentences:
        result = phonemizer.phonemise_list([text], lang="deu", expand_acronyms=False)
        

        
        print("_" * 100)

        for text_word, pred in result.predictions.items():
            tokens, probs = pred.phoneme_tokens, pred.token_probs
            tokens = ''.join(tokens)
            print(f'{text_word} | {tokens} | {pred.confidence}')

        print(text)
        print(result.phonemes)


In [ ]:
from glob import glob

duden_lines = []
with open(
    "/home/DeepPhonemizer/tsv/deu_duden.tsv",
    "r",
    encoding="utf-8",
) as rf:
    for line in rf:
        duden_lines.append(line)


duden_words = [line.split("\t")[0].strip() for line in duden_lines]


wikipron_lines = []
with open(
    "/home/DeepPhonemizer/tsv-extra/deu_latn_narrow.tsv",
    "r",
    encoding="utf-8",
) as rf:
    for line in rf:
        wikipron_lines.append(line)

wikipron_words = [line.split("\t")[0].strip() for line in wikipron_lines]

print(len(wikipron_words))

# filter out words from wikipron that are in duden
wikipron_words = set(wikipron_words) - set(duden_words)

# now iterate over wikipron_lines and filter out the words that are not in wikipron_words
wikipron_lines = [line for line in wikipron_lines if line.split("\t")[0].strip() in wikipron_words]


# write the filtered wikipron_lines to a new file
with open(
    "/home/DeepPhonemizer/tsv-extra/deu_latn_narrow_filtered.tsv",
    "w",
    encoding="utf-8",
) as wf:
    for line in wikipron_lines:
        wf.write(line)